# 감성분석 다른말로 이진분류
# IMDB영화평 감성분석 (이진분류)

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t')
df.head(3)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


In [20]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3) # 3 :  Quote - None
df.head(3)
# 쌍따옴표 있는채로 데이터들어와.

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [21]:
df.info() #2만 5000개 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [22]:
df.review[0][:1000] 
# 리뷰의 앞에서부터 1000번째글자 까지 ...

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리 

In [ ]:
# <br/> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />', ' ')
# df.review[0][:1000]

In [28]:
# 구둣점하고 숫자 제거 - 영문자가 아닌놈들은 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()
df.review[0][:1000]

<ipython-input-28-59c6a74fac11>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

In [ ]:
# dir(str) str메소드. 

- Train / test set 분리 

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape
y_train.value_counts()


0    9375
1    9375
Name: sentiment, dtype: int64

In [34]:
y_test.value_counts()

0    3125
1    3125
Name: sentiment, dtype: int64

- CountVectorizer로 변환
    - 잘들으세요 .
    - X_train, x_test 비율 0.25니까 3:1 이야
    - 20000  대 10000에 동의한다면. 
    인덱스 19999 대 9999 
    만약 at이라는 단어가 인덱스가 100번째 에서 50번째로 변경 (단어가 별로 없어서 인덱스 당겨진다)

    그러므로 트레인 테스트일때, 

    X _ train  fit 


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [41]:
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [42]:
# 틀리는 방법
cvect2 = CountVectorizer(stop_words='english')
X_test_cv = cvect2.fit_transform(X_test)  ## 트래인 핏한 놈으로 핏 트랜스폼
X_test_cv.shape
# 테스트 셋의 열이 달라 ㅠ 실패

(6250, 41343)

In [43]:
def get_word(index, voca):
    for key, value in voca.items():
        if value == index:
            return key

get_word(100, cvect.vocabulary_), get_word(100, cvect2.vocabulary_)

('abilityof', 'abos')

위는 어떤 의미냐면은 훈련은 100번째 abilityof 에 대해서 해줬는데, 테스트는 100번째 abos 에 대해서 하는 꼴.. cvect2 정정해줘야 
어떤 문제가 있냐 또 원래 트레인 데이터에는 없지만. 테스트 데이터에만 있는 문자 단어도 있어 그러나 대세에 영향을 미치지 못한다

In [44]:
# 테스트 데이ㅓ셋은 train dataset 을 변환한 Vectorizer로 반드시 변환하여야 함.
X_test_cv = cvect.transform(X_test)
X_test_cv.shape
# (18750, 65213) 트래인 세트의 쉐입 과 열을 비교해야되. 

(6250, 65213)

In [48]:
print(dir(str))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


- 분류기 - 로지스틱 회귀

In [52]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression(random_state=2022, max_iter=500)
lr.fit(X_train_cv, y_train)   # cv와 y_train 으로 핏 주의

LogisticRegression(max_iter=500, random_state=2022)

In [53]:
lr.score(X_test_cv, y_test)

0.86576

- CountVectorizer + ngaram_range = (1,2) 로 바꿔서 해보자 

In [57]:
cvect12 = CountVectorizer(stop_words='english', ngram_range=(1,2))

In [58]:
cvect12.fit(X_train) # x_train 데이터로 학습
%time X_train_cv12 = cvect12.transform(X_train)


Wall time: 6.51 s


In [59]:
X_test_cv12 = cvect12.transform(X_test)
X_train_cv12.shape, X_test_cv12.shape
# 바이그램 추가후 쉐입 확인 ((18750, 1386558), (6250, 1386558))
# 기존 모노그램 버전은 (6250, 65213) 이정도.

((18750, 1386558), (6250, 1386558))

In [62]:
lr12 = LogisticRegression(max_iter=500, random_state=2022)
lr12.fit(X_train_cv12, y_train)
lr12.score(X_test_cv12, y_test)
# 0.86576 에서 0.87952 으로 정확도 상승 바이그램 버전

0.87952

- 또 뭘 더해볼 수 있을까? - c벡터 모델 12 등등 저장하는것 가능해

In [63]:
import joblib


In [66]:
#모델 저장 
joblib.dump(cvect12, 'model/imdb_cvect12.pkl')
joblib.dump(lr12, 'model/lmdb_cvect_lr12.pkl')

['model/lmdb_cvect_lr12.pkl']

In [67]:
# 모델 로딩 
cvect12 = joblib.load('model/imdb_cvect12.pkl')
lr12 = joblib.load('model/lmdb_cvect_lr12.pkl')

- 실제DB로검증

In [69]:
review = '''
What cinemas were made for. I wasn't expecting something quite as amazing as this, this was two and a half hours of incredible entertainment, drama, laughs, tears and action galore, there truly was something for everyone here.

This movie really has ignited my love for the franchise once again, and I am truly excited for what comes next.

Nostalgia, there seems to be a real thirst for it theses days, and there is a lot of it to enjoy here. There are plenty of moments throughout that will have you speechless, they really do break all of the rules here.

This is one incredibly good looking movie, dazzling special effects, dizzying action scenes, two and a half hours will literally fly by.

Incredible, 10/10.
'''
review

"\nWhat cinemas were made for. I wasn't expecting something quite as amazing as this, this was two and a half hours of incredible entertainment, drama, laughs, tears and action galore, there truly was something for everyone here.\n\nThis movie really has ignited my love for the franchise once again, and I am truly excited for what comes next.\n\nNostalgia, there seems to be a real thirst for it theses days, and there is a lot of it to enjoy here. There are plenty of moments throughout that will have you speechless, they really do break all of the rules here.\n\nThis is one incredibly good looking movie, dazzling special effects, dizzying action scenes, two and a half hours will literally fly by.\n\nIncredible, 10/10.\n"

In [72]:
# 리플레이스 자체는 정규표현식 사용할 수 없어. 
# re 모듈 임포트 할것.
review.replace('[^A-Za-z]',' ')

"\nWhat cinemas were made for. I wasn't expecting something quite as amazing as this, this was two and a half hours of incredible entertainment, drama, laughs, tears and action galore, there truly was something for everyone here.\n\nThis movie really has ignited my love for the franchise once again, and I am truly excited for what comes next.\n\nNostalgia, there seems to be a real thirst for it theses days, and there is a lot of it to enjoy here. There are plenty of moments throughout that will have you speechless, they really do break all of the rules here.\n\nThis is one incredibly good looking movie, dazzling special effects, dizzying action scenes, two and a half hours will literally fly by.\n\nIncredible, 10/10.\n"

In [70]:
# 텍스트 전처리 영문자 이외에는 공백 블랭크 처리. br은 없는 버전
import re
clean_review = re.sub('[^A-Za-z]',' ', review).strip()

In [73]:
clean_review
# type(clean_review)

str

In [74]:
# 변환을 해야 해 클린리뷰를 c벡터 12로 변환 

review_cv = cvect12.transform([clean_review])

In [75]:
review_cv.shape

(1, 1386558)

In [77]:
lr12.predict(review_cv)

array([1], dtype=int64)